# SQL SubQuery
## How to load the dataset(s) to SQL?
Using Python, you can load the large datasets to MySQL database very easily. For that follow the below steps.

- First create a database in your local machine server.

```sql
CREATE DATABASE <database_name>
```

- Next, use Python to load the database

```python
import pandas as pd
from sqlalchemy import create_engine

df = pd.read_csv("file/path/to/the/database.csv")

engine = create_engine("mysql+pymysql://<db_username>:<db_password>@<hostname>/<database_name>")
df.to_sql("<table_name>", con=engine)
```

## Problems 1-6

For problems 1 to 6, use the Olympic dataset. You can get that from [here](https://drive.google.com/file/d/1EGIRBkbQGByJPvCqDtxtTnXv93oGunFp/view?usp=share_link).

**Column description:**
1. ID -> ID of every records to our dataset. It has integer datatype.
2. Name -> Name of the athletes.
3. Sex -> Gender of the athletes.
4. Height -> Height of the athletes
5. Weight -> Weight of the athletes
6. NOC -> In which country, the athletes belong to. This is actually the country code.
7. Year -> In which year, the athlete has participated
8. Sport -> What is the sport name in which the athlete participated.
9. Event -> Event name of the sport
10. Medal -> Which medal the athlege got. If the athlete did not get any medal then this cell is blank.
11. country -> The name of the country.

### Problem 1

Display the names of athletes who won a gold medal in the 2008 Olympics and whose height is greater than the average height of all athletes in the 2008 Olympics.

SELECT * FROM olympics
WHERE Year = 2008 AND
Medal = 'Gold' AND
Height > (SELECT AVG(Height) FROM olympics WHERE Year = 2008);



### Problem 2

Display the names of athletes who won a medal in the sport of basketball in the 2016 Olympics and whose weight is less than the average weight of all athletes who won a medal in the 2016 Olympics.

SELECT name FROM olympics
WHERE Year = 2016 AND
Sport = 'Basketball' AND 
Medal IS NOT NULL AND
height < (SELECT AVG(Height) FROM olympics WHERE Year = 2016
		  AND Medal IS NOT NULL);


### Problem 3

Display the names of all athletes who have won a medal in the sport of swimming in both the 2008 and 2016 Olympics.

SELECT Name FROM olympics
WHERE Sport = 'Swimming'
  AND Year IN (2008, 2016)
  AND Medal IS NOT NULL
GROUP BY Name
HAVING COUNT(DISTINCT Year) = 2
ORDER BY Name;

# Using Sub Query:

SELECT DISTINCT Name
FROM olympics
WHERE Sport = 'Swimming'
  AND Medal IS NOT NULL
  AND Name IN (
    SELECT Name
    FROM olympics
    WHERE Sport = 'Swimming' AND Year = 2008 AND Medal IS NOT NULL
  )
  AND Name IN (
    SELECT Name
    FROM olympics
    WHERE Sport = 'Swimming' AND Year = 2016 AND Medal IS NOT NULL
  ) ORDER BY Name;

  # Using Inner Join:

SELECT Distinct(t1.Name) FROM (SELECT Name FROM olympics
WHERE Year = 2016 AND Sport = 'Swimming' AND Medal IN ('Gold','Silver','Bronze')) t1
INNER JOIN
(SELECT Name FROM olympics
WHERE Year = 2008 AND Sport = 'Swimming' AND Medal IN ('Gold','Silver','Bronze')) t2
ON t1.Name = t2.Name
Order By Name;

### Problem 4

Display the names of all countries that have won more than 50 medals in a single year.

SELECT country,Year,COUNT(*) FROM olympics
WHERE Medal IS NOT NULL AND country IS NOT NULL
GROUP BY country,Year
HAVING COUNT(*) > 50
ORDER BY Year,country;



### Problem 5

Display the names of all athletes who have won medals in more than one sport in the same year.

SELECT DISTINCT name FROM olympics
WHERE ID in (SELECT DISTINCT ID FROM olympics
			WHERE Medal IS NOT NULL
			GROUP BY ID,Year,Sport
			HAVING COUNT(Medal) > 1
			ORDER BY COUNT(Medal) DESC);

### Problem 6

What is the average weight difference between male and female athletes in the Olympics who have won a medal in the same event?

WITH result AS (
	SELECT * FROM olympics
	WHERE Medal IS NOT NULL
)
SELECT AVG(A.Weight - B.Weight) FROM result A
JOIN result B
ON A.Event = B.Event
ANd A.Gender != B.Gender;




## Problem 7 - 10

Use the health insurance dataset. You can get the dataset as well as the description of the dataset [here](https://www.kaggle.com/datasets/thedevastator/insurance-claim-analysis-demographic-and-health).

### Problem 7

How many patients have claimed more than the average claim amount for patients who are smokers and have at least one child, and belong to the southeast region?

SELECT COUNT(claim) FROM insurance
WHERE claim > (SELECT AVG(Claim) FROM insurance
				WHERE smoker = 'Yes' AND
				region = 'southwest' AND
				children >= 1); 


### Problem 8

How many patients have claimed more than the average claim amount for patients who are not smokers and have a BMI greater than the average BMI for patients who have at least one child?

SELECT COUNT(claim) FROM insurance
WHERE claim > (SELECT AVG(claim) FROM insurance
				WHERE smoker = 'No' AND
				bmi > (SELECT AVG(bmi) FROM insurance
						WHERE children >= 1));

### Problem 9

How many patients have claimed more than the average claim amount for patients who have a BMI greater than the average BMI for patients who are diabetic, have at least one child, and are from the southwest region?

SELECT COUNT(claim) FROM insurance
WHERE claim > (SELECT AVG(claim) FROM insurance WHERE
			   bmi > (SELECT AVG(bmi) FROM insurance
					  WHERE children >= 1 AND
                      diabetic = 'Yes' AND
                      region = 'southwest'));




### Problem 10:

What is the difference in the average claim amount between patients who are smokers and patients who are non-smokers, and have the same BMI and number of children?

SELECT bmi, children, AVG(claim) AS smoker_avg_claim, (
    SELECT AVG(claim)
    FROM insurance_data AS non_smoker
    WHERE non_smoker.bmi = smoker.bmi
    AND non_smoker.children = smoker.children
    AND non_smoker.smoker = 'No'
) AS non_smoker_avg_claim, AVG(claim) - (
    SELECT AVG(claim)
    FROM insurance_data AS non_smoker
    WHERE non_smoker.bmi = smoker.bmi
    AND non_smoker.children = smoker.children
    AND non_smoker.smoker = 'No'
) AS claim_diff
FROM insurance_data AS smoker
WHERE smoker.smoker = 'Yes'
GROUP BY smoker.bmi, smoker.children
having claim_diff is not null
ORDER BY bmi, children;

